__Objective__: Computing cross-validation score of CMV dataset using BERT

__Runtime__: GPU

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install transformers 
!pip install datasets

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm 
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib import colors

import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_metric

# Loading CMV dataset and creating folds

In [ ]:
def read_dataset(dir):
    """Reading texts and labels from dataset"""
    texts_labels = []
    with open(dir, 'r', encoding='utf-8') as f:
        lines = f.readlines()[1:]
        for line in lines:
            line = line.strip().split(",")
            assert(len(line) == 2)
            label = line[0].strip()
            text = line[1].strip()
            if label == 'AH':
                texts_labels.append((text, 1))
            else:
                texts_labels.append((text, 0))
    return texts_labels

In [ ]:
dataset = read_dataset('/content/gdrive/MyDrive/DL/dataset/pytorch/train.csv')
dataset.extend(read_dataset('/content/gdrive/MyDrive/DL/dataset/pytorch/test.csv'))

In [ ]:
len(dataset)

In [ ]:
def split_and_merge(dataset):
    ah = []
    none = [] 
    for ctext, clabel in dataset:
        if (clabel):
            ah.append((ctext, clabel)) 
        else:
            none.append((ctext, clabel)) 
    i = 0 
    j = 0 
    new_dataset = []
    while (i < len(ah) and j < len(none)):
        new_dataset.append(ah[i]) 
        new_dataset.append(none[j])
        i += 1
        j += 1
    while (i < len(ah)):
        new_dataset.append(ah[i]) 
        i += 1 
    while (j < len(none)): 
        new_dataset.append(none[j]) 
        j += 1 
    return new_dataset

In [ ]:
# shuffling the dataset 
dataset = split_and_merge(dataset)

# creating folds 
n_folds = 10
fold_length = len(dataset) // n_folds
folds = [dataset[i * fold_length: (i + 1) * fold_length] for i in range(n_folds)]

In [ ]:
# creating tokenizer to get encodings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') 
max_seq_length = 64

In [ ]:
folds_text = [] 
folds_label = [] 
for i in range(n_folds):
    texts = []
    labels = [] 
    for ftext, flabel in folds[i]:
        texts.append(ftext)
        labels.append(flabel)
    folds_text.append(texts)
    folds_label.append(labels) 

In [ ]:
# generating encodings
folds_encoding = [] 
for i in range(n_folds):
    folds_encoding.append(tokenizer(folds_text[i], truncation=True, max_length=max_seq_length, padding="max_length"))

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = []
test_dataset = [] 

for i in tqdm(range(n_folds)):
    cur_text = [] 
    cur_label = [] 
    for j in range(n_folds):
        if i == j:
            continue 
        cur_text.extend(folds_text[j]) 
        cur_label.extend(folds_label[j]) 
    cur_encoding = tokenizer(cur_text, truncation=True, max_length=max_seq_length, padding="max_length")
    train_dataset.append(CustomDataset(cur_encoding, cur_label)) 
    test_dataset.append(CustomDataset(folds_encoding[i], folds_label[i]))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
bert_acc = [] 
bert_f1 = []

In [ ]:
from copy import deepcopy

In [ ]:
def evaluate(fold_id):
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
    model.to(device)
    model.train()

    train_loader = DataLoader(train_dataset[fold_id], batch_size=64, shuffle=True)
    optim = AdamW(model.parameters(), lr=5e-5)

    for epoch in range(3):
        for batch in tqdm(train_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            loss.backward()
            optim.step()

    acc = load_metric("accuracy")
    f1 = load_metric("f1")
    model.eval()
    eval_loader = DataLoader(test_dataset[fold_id], batch_size=64, shuffle=False)
    for batch in eval_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        acc.add_batch(predictions=predictions, references=batch["labels"])
        f1.add_batch(predictions=predictions, references=batch["labels"])
    
    acc_ = deepcopy(acc.compute())
    f1_ = deepcopy(f1.compute())
    bert_acc.append(acc_)
    bert_f1.append(f1_)
    print(acc_)
    print(f1_)

In [ ]:
for i in range(10):
    evaluate(i)
    print(f'\n\n\n{"-" * 100}\n\n\n')

In [ ]:
lst_acc = [] 
lst_f1 = [] 

for i in range(10):
    lst_acc.append(bert_acc[i]['accuracy'])
    lst_f1.append(bert_f1[i]['f1'])

In [ ]:

df = pd.DataFrame({'Accuracy': lst_acc, 'F1': lst_f1})

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.boxplot()